In [ ]:
%matplotlib notebook
# interactive notebook format is required for the interactive plot

import numpy as np
import pandas as pd
import math
import random

from sklearn.decomposition import PCA
from sklearn.manifold import MDS
import sklearn.metrics.pairwise

from sklearn.manifold import TSNE
import functools
import time

import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [ ]:
filename, norm = 'Animal_Data_Andromeda.csv', False  # do not normalize animal dataset, all columns are 0-100 scale
#filename, norm = 'InfoVis_Fall_2015_Survey.csv', True

df = pd.read_csv(filename)

# Use 'Name' column as index
#df.rename(columns={df.columns[0]:'Name'}, inplace=True)
df.set_index('Name', inplace=True)

# Sort rows and columns
df.sort_index(axis=1, inplace=True)
df.sort_index(inplace=True)

# split data into two groups, we are focused on using numerical data only
df_numeric = df.select_dtypes(include='number')  #'int32' or 'int64' or 'float32' or 'float64'
df_category = df.select_dtypes(exclude='number') #'object'

# Z-score normalization
normalized_df = df_numeric
if norm:
    normalized_df = (df_numeric - df_numeric.mean()) / df_numeric.std()

print('Data size (r,c) =', normalized_df.shape)
df_numeric.head(2)

In [ ]:
# Compute the distance matrix for the weighted high-dimensional data using L1 distance function.
#  Input HD data should already be weighted.
def distance_matrix_HD(dataHDw):  # dataHDw (pandas or numpy) -> distance matrix (numpy)
    dist_matrix = sklearn.metrics.pairwise.manhattan_distances(dataHDw)
    #m = pd.DataFrame(m, columns=dataHD.index, index=dataHD.index)  # keep as np array for performance
#     dist_matrix = sklearn.metrics.pairwise.euclidean_distances(dataHDw)
    return dist_matrix

# Compute the distance matrix for 2D projected data using L2 distance function.
def distance_matrix_2D(data2D):  # data2d (pandas or numpy) -> distance matrix (numpy)
    dist_matrix = sklearn.metrics.pairwise.euclidean_distances(data2D) 
    #m = pd.DataFrame(m, columns=data2D.index, index=data2D.index) # keep as np array for performance
    return dist_matrix

#def dist(x,y):
#    return np.linalg.norm(x-y, ord=2)


# Calculate the MDS stress metric between HD and 2D distances.  Uses numpy for efficiency.
def stress(distHD, dist2D):  #  distHD, dist2D (numpy) -> stress (float)
    #s = np.sqrt((distHD-dist2D).pow(2).sum().sum() / distHD.pow(2).sum().sum())  # pandas
    #s = np.sqrt(((distHD-dist2D)**2).sum() / (distHD**2).sum())   # numpy
    s = ((distHD-dist2D)**2).sum() / (distHD**2).sum()   # numpy, eliminate sqrt for efficiency
    return s

In [ ]:
def compute_mds(dataHDw):  # dataHDw -> data2D (pandas)
    distHD = distance_matrix_HD(dataHDw)
    
#     # Adjust these parameters for performance/accuracy tradeoff
    mds = sklearn.manifold.MDS(n_components=2, dissimilarity='precomputed', n_init=10, max_iter=1000)
#     # Reduction algorithm happens here:  data2D is nx2 matrix
    data2D = mds.fit_transform(distHD)
    
    pca = sklearn.decomposition.PCA(n_components=2)
    pca_res = pca.fit_transform(distHD)
    # Rotate the resulting 2D projection to make it more consistent across multiple runs.
    # Set the 1st PC to the y axis, plot looks better to spread data vertically with horizontal text labels
    pca = sklearn.decomposition.PCA(n_components=2)
    data2D = pca.fit_transform(data2D)
    data2D = pd.DataFrame(data2D, columns=['y','x'], index=dataHDw.index)
    
    data2D.stress_value = stress(distHD, distance_matrix_2D(data2D))
    return data2D

def compute_pca(dataHDw): #dataHDw -> data2D (pandas)
    scaled_data = sklearn.preprocessing.scale(dataHDw)
    # Builds the PCA model to get 2 principal components to graph
    pca = sklearn.decomposition.PCA(n_components=2)
    pca.fit(scaled_data)
    pca_data = pca.transform(scaled_data)
    data_2D = pd.DataFrame(pca_data, columns=['x', 'y'], index=dataHDw.index)
    percentage_variation = np.round(pca.explained_variance_ratio_ * 100, decimals = 2)
#     print(percentage_variation)
    return data_2D

def compute_tsne(dataHDw):  # dataHDw -> data2D
    distHD = distance_matrix_HD(dataHDw)
    # We want to get TSNE embedding with 2 dimensions
    tsne = TSNE(n_components=2, perplexity=10, metric="euclidean", verbose=0, learning_rate='auto', init='pca', n_iter=10000)
    tsne_result = tsne.fit_transform(dataHDw) # fit param into embedded space, return that transformed output
    # Rotate the resulting 2D projection to make it more consistent across multiple runs.
    # Set the 1st PC to the y axis, plot looks better to spread data vertically with horizontal text labels
    pca = sklearn.decomposition.PCA(n_components=2)
    tsne_data2D = pca.fit_transform(tsne_result)
    tsne_data2D = pd.DataFrame(tsne_result, columns=['y','x'], index=dataHDw.index)

    tsne_data2D.stress_value = stress(distHD, distance_matrix_2D(dataHDw))
    return tsne_data2D

In [ ]:
def dimension_reduction(dataHD, wts, dr_algo="mds"): # dataHD, wts -> data2D (pandas)
    # Normalize the weights to sum to 1
    wts = wts/wts.sum()
    # Apply weights to the HD data
    dataHDw = dataHD * wts
    # DR algorithm
    if dr_algo == "mds":
        start = time.time()
        data2D = compute_mds(dataHDw)
        end = time.time()
    elif dr_algo == "pca":
        start = time.time()
        data2D = compute_pca(dataHDw)
        end = time.time()
    else: # assume tSNE
        start = time.time()
        data2D = compute_tsne(dataHDw)
        end = time.time()
    # Compute row relevances as:  data dot weights
    # High relevance means large values in upweighted dimensions
    data2D['relevance'] = dataHDw.sum(axis=1)
    return data2D, end-start

# Setup weights
min_weight, max_weight = 0.00001, 0.9999
init_weight = min_weight  # 1.0/len(normalized_df.columns) # initialize to min to make the sliders easier to use.
weights = pd.Series(init_weight, index=normalized_df.columns, name="Weight")  # the current weight list
# Compute projected data
run_times = {
    "pca": 0,
    "mds": 0,
    "tsne": 0,
}
tsne_df_2D, run_times["tsne"] = dimension_reduction(normalized_df, weights, "tsne") 
df_2D, run_times["mds"] = dimension_reduction(normalized_df, weights, "mds") 
pca_df_2D, run_times["pca"] = dimension_reduction(normalized_df, weights, "pca")


In [ ]:
weights.head(2)

In [ ]:
pd.DataFrame(distance_matrix_HD(normalized_df * (weights/weights.sum())), 
             columns=normalized_df.index, index=normalized_df.index).head(2)

In [ ]:
#print("MDS Stress Value: ", df_2D.stress_value)
df_2D.head(2)
# print(tsne_df_2D.stress_value) # calculate KL divergence(?)
# tsne_df_2D.head(2)

In [ ]:
# This method is used to propose a new weight for current column in a smart fashion
def new_proposal(current, step, direction):
    return np.clip(current + direction*step*random.random(), 0.00001, 0.9999)

# Repeatedly tries to modify each dim weight to see if it improves the stress, thus
# getting the weighted high-dim distances to more closely match the input 2D distances.
# A simplified gradient descent that should work with various distance functions.
#   dataHD = high-dim data, as pandas
#   data2D = 2D data input, as pandas
#   weights = as pandas series, or None for weights[i]=1/p
def inverse_DR(dataHD, data2D, curWeights=None):  # -> new weights, as Series
    dist2D = distance_matrix_2D(data2D)  # compute 2D distances only once
    col_names = dataHD.columns
    dataHD = dataHD.to_numpy()  # use numpy for efficiency
    row, col = dataHD.shape
    
    if curWeights==None:
        curWeights = np.array([1.0/col]*col)  # default weights = 1/p
    else:
        curWeights = curWeights.to_numpy()
        curWeights = curWeights / curWeights.sum()  # Normalize weights to sum to 1
    newWeights = curWeights.copy()  # re-use this array for efficiency
    
    # Initialize state
    flag = [0]*col         # degree of success of a weight change
    direction = [1]*col  # direction to move a weight, pos or neg
    step = [1.0/col]*col   # how much to change each weight
    
    dataHDw = dataHD * curWeights   # weighted space, re-use this array for efficiency
    distHD = distance_matrix_HD(dataHDw)
    curStress = stress(distHD, dist2D)
    print('Starting stress =', curStress, 'Processing...')

    MAX = 500   # default setting of the number of iterations

    # Try to minorly adjust each weight to see if it reduces stress
    for i in range(MAX):
        for dim in range(col):            
            # Get a new weight for current column
            nw = new_proposal(curWeights[dim], step[dim], direction[dim])
            
            # Scale the weight list such that it sums to 1
            #newWeights = curWeights.copy()  # avoid extra copy op using math below
            #newWeights[dim] = nw
            #newWeights = newWeights / s
            s = 1.0  + nw - curWeights[dim]   # 1.0 == curWeights.sum()
            np.true_divide(curWeights, s, out=newWeights)  # transfers to other array, while doing /
            newWeights[dim] = nw / s
            
            # Apply new weights to HD data
            np.multiply(dataHD, newWeights, out=dataHDw)  # dataHDw = dataHD * newWeights; efficiently reuses dataHDw array
            distHD = distance_matrix_HD(dataHDw)

            # Get the new stress
            newStress = stress(distHD, dist2D)
            
            # If new stress is lower, then update weights and flag this success
            if newStress < curStress:
                temp = curWeights
                curWeights = newWeights
                newWeights = temp   # reuse the old array next iteration
                curStress = newStress
                flag[dim] = flag[dim] + 1
            else:
                flag[dim] = flag[dim] - 1
                direction[dim] = -direction[dim]  # Reverse course
    
            # If recent success, then speed up the step rate
            if flag[dim] >= 5:
                step[dim] = step[dim] * 2
                flag[dim] = 0
            elif flag[dim] <= -5:
                step[dim] = step[dim] / 2
                flag[dim] = 0
                
    print('Solution stress =', curStress, 'Done.')
    #print("weight", curWeights)
    #print("flag", flag)
    #print("dir", direction)
    #print("step", step)
    return pd.Series(curWeights, index=col_names, name="Weight")


In [ ]:
def create_sliders(wts):
    # Create sliders, one for each dimension weight
    sliders = [widgets.FloatSlider(min=min_weight, max=max_weight, step=0.01, value=value, 
                                       description=label, continuous_update=False, readout_format='.5f')
                   for (label, value) in wts.iteritems()]
    # Display sliders
#     for s in sliders:
#         #s.observe(sliderchange, names='value')
#         display(s)
    # Make list len even
    if len(sliders)%2 == 1:
        sliders.append(widgets.Label(value=""))
    # Show two sliders in each row to make more vertically compact
    for i in range(0, len(sliders), 2):
        slider_row = widgets.HBox([sliders[i], sliders[i+1]])
        display(slider_row)
        
    #create_slider_buttons(sliders)
    return sliders

def create_slider_buttons(sliders):
    apply_button = widgets.Button(description='Apply Slider Weights (MDS)', layout=widgets.Layout(width='auto'))
    tsne_apply_button = widgets.Button(description='Apply Slider Weights (tSNE)', layout=widgets.Layout(width='auto'))
    pca_apply_button = widgets.Button(description='Apply Slider Weights (PCA)', layout=widgets.Layout(width='auto'))
    reset_button = widgets.Button(description='Reset Plots')

    # Callback functions
    def apply_button_clicked(change, dr_="mds"):
        # Use the slider values to re compute the DR and redraw the plot
        global weights, df_2D   # Update weights and df_2D globals
        weights = pd.Series([s.value for s in sliders], index=normalized_df.columns, name='Weight')
        if dr_ == "mds":
            df_2D, run_times["mds"] = dimension_reduction(normalized_df, weights, "mds")
            # Re-draw the plot
            draw_plot(plot_ax, df_2D)
            print("Elapsed Time: %.4f"%(run_times["mds"]))
        elif dr_ == "tsne":
            tsne_df_2D, run_times["tsne"] = dimension_reduction(normalized_df, weights, "tsne")
            # Re-draw the plot
            draw_plot(plot_ax2, tsne_df_2D)
            print("Elapsed Time: %.4f"%(run_times["tsne"]))
        else: # assume pca
            pca_df_2D, run_times["pca"] = dimension_reduction(normalized_df, weights, "pca")
            # Re-draw the plot
            draw_plot(plot_ax3, pca_df_2D)
            print("Elapsed Time: %.4f"%(run_times["pca"]))
    apply_button.on_click(functools.partial(apply_button_clicked, dr_="mds"))
    tsne_apply_button.on_click(functools.partial(apply_button_clicked, dr_="tsne"))
    pca_apply_button.on_click(functools.partial(apply_button_clicked, dr_="pca"))
    apply_buttons = widgets.HBox([apply_button, tsne_apply_button, pca_apply_button])

    def reset_button_clicked(change):
        # Reset all sliders to initial value and re-compute DR and re-draw the plot
        for s in sliders:
            s.value = init_weight
        apply_button_clicked(change, dr_="mds")
        apply_button_clicked(change, dr_="tsne")
        apply_button_clicked(change, dr_="pca")
    reset_button.on_click(reset_button_clicked)
    
    # Display buttons
    display(apply_buttons)
    display(reset_button)
    return apply_button, tsne_apply_button, pca_apply_button, reset_button


In [ ]:
def create_detail_display():
    # Print selected points
    print_button = widgets.Button(description='Print selected points (MDS plot)', layout=widgets.Layout(width='auto'))
    print_output = widgets.Output()
    tsne_print_button = widgets.Button(description='Print selected points (tSNE plot)', layout=widgets.Layout(width='auto'))
    pca_print_button = widgets.Button(description='Print selected points (PCA plot)', layout=widgets.Layout(width='auto'))

    def print_button_clicked(change, dr_="mds"):
        print_output.clear_output()
        # Get list of selected points and print their source data values
        circles = None
        if dr_=="mds":
            circles = plot_ax.circles
        elif dr_=="tsne":
            circles = plot_ax2.circles
        else:
            circles = plot_ax3.circles
        selset = [c.index for c in (circles) if c.selected]
        with print_output:
            if len(selset) > 0:
                print(df.iloc[selset, :].transpose())
            else:
                print('Select points in the plot to see details here')
    print_button.on_click(print_button_clicked)
    tsne_print_button.on_click(functools.partial(print_button_clicked, dr_="tsne"))
    pca_print_button.on_click(functools.partial(print_button_clicked, dr_="pca"))
    
    print_buttons = widgets.HBox([print_button, tsne_print_button, pca_print_button])
    display(print_buttons)
    display(print_output)
    return print_button, print_output
